Below is the model for Fake News detection over the Buzzfeed-Webis Fake News Corpus 2016. 

In [93]:
np.random.seed(1337) # Fix a random seed to make reproducible results

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
import numpy as np

import xml.etree.ElementTree as ET
import os
from gensim.models import Word2Vec, KeyedVectors

# CNN - Second Implementation (Medium post)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Embedding, Input, Conv1D, MaxPooling1D, Dropout
from keras.utils import to_categorical
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.regularizers import l2

The first step is to download the Buzzfeed-Webis corpus, which is provided in the form of XML files. read_files will read each file and parse the XML tree to retrieve a tuple of the body of the text ('mainText') and the veracity label ('veracity')

In [55]:
def read_files(cols):
    """
    For each xml file return a matrix of values asked for
    """
    path = 'data/train/'
    possibilities = ['mixture of true and false', 'mostly false', 'no factual content', 'mostly true']
    for filename in os.listdir(path):
        data_row = []
        if not filename.endswith('.xml'): continue
        xmlfile = os.path.join(path, filename)
        tree = ET.parse(xmlfile)
        if cols == "mainText":
            if tree.find("mainText").text:
                yield tree.find("mainText").text
            else:
                yield ''
        elif cols == "veracity":
            v = possibilities.index(tree.find("veracity").text)
            yield v
        elif cols == "both":
            if tree.find("mainText").text:
                v = possibilities.index(tree.find("veracity").text)
                yield tree.find("mainText").text, v
            else:
                yield None,None
        else:
            for col in cols:
                try:
                    data_row.append(int(tree.find(col).text))
                except:
                    data_row.append(0)
            yield data_row

We call this function to get a list of the main text of each article ('documents') as well as a matching list of the labels ('predictions')

In [56]:
def feature_matrix(cols):
    data = []
    for row in read_files(cols):
        data.append(row)
    return np.array(data)

def get_document_text():
    data = []
    for row in read_files("mainText"):
        if not row:
            continue
        else:
            data.append(row)
    return data

def get_veracity():
    data = []
    for row in read_files("veracity"):
        data.append(row)
    return data

def get_document_text_and_veracity():
    docs, preds = [], []
    for row in read_files("both"):
        if not row[0]:
            continue
        else:
            docs.append(row[0])
            preds.append(row[1])
    return docs, preds

documents, predictions = get_document_text_and_veracity()

Now we load the Google News pre-trained word embeddings for use in our model. These embeddings are trained using a combination of CBOW and skip-grams over a corpus of over 100 billion words from Google News.  

In [57]:
file = 'data/GoogleNews-vectors-negative300.bin'
embeddings = KeyedVectors.load_word2vec_format(file, binary=True)

To represent entire articles using the Google News word embeddings, we replace each string with it's matching embedding and then taken the elementwise mean, max or min of the entire document. This takes a document of N words from being N separate vectors to being a single 1D vector (300 by 1)

In [58]:
# Note: Is there a more efficient way to do this? Looping might be slow for long articles
def avg_docvec(docText,embeddings):
    """
    This function converts the text of a document (input as a string) to word embeddings, then
    takes the elementwise average of the embeddings to return a single vector.
    """
    docVec = np.zeros(300) # Initialize array for the document
    tokens = word_tokenize(docText) # Creates a list of word tokens (e.g. "Test words" -> ['Test', 'words'])
    denominator = 0.0 # To take the average, will only count tokens for which we have embeddings in the total  
    for token in tokens:
        try:
            v = embeddings[token]
            np.add(docVec,v,out=docVec)
            denominator += 1.0
        except: # Ignore tokens that aren't in the Google News embeddings
            continue
    np.divide(docVec,denominator,out=docVec) 
    return docVec

In [59]:
def max_docvec(docText,embeddings):
    """
    Converts the text of a document (input as a string) to word embeddings, then takes the elementwise
    max of the embeddings to return a single vector of the maximum elements.
    """
    docVec = 0
    tokens = word_tokenize(docText) # Creates a list of word tokens (e.g. "Test words" -> ['Test', 'words'])
    startIndex = 0
    for i in range(len(tokens)): # Initialize the doc vec as the first token that is in the embeddings
        try:
            v = embeddings[tokens[i]]
            docVec = v
            startIndex = i
            break
        except:
            continue
    
    for token in tokens[startIndex:]:
        try:
            v = embeddings[token]
            np.max(docVec,v,out=docVec)
        except: # Ignore tokens that aren't in the Google News embeddings
            continue
    return docVec

In [60]:
def min_docvec(docText,embeddings):
    """
    Converts the text of a document (input as a string) to word embeddings, then takes the elementwise
    min of the embeddings to return a single vector of the minimum elements.
    """
    docVec = 0
    tokens = word_tokenize(docText) # Creates a list of word tokens (e.g. "Test words" -> ['Test', 'words'])
    startIndex = 0
    for i in range(len(tokens)): # Initialize the doc vec as the first token that is in the embeddings
        try:
            v = embeddings[tokens[i]]
            docVec = v
            startIndex = i
            break
        except:
            continue
    for token in tokens[startIndex:]: # Loop over words in the article, starting at first valid word
        try:
            v = embeddings[token]
            np.min(docVec,v,out=docVec) # Only keep min elements
        except: # Ignore tokens that aren't in the Google News embeddings
            continue
    return docVec

In [61]:
def docs_to_matrix(documents,embeddings,method='avg'):
    """
    Takes a list of document text strings and returns a matrix of document embeddings.
    The method specifies how the word vectors are combined for the document: average is 
    element-wise average, min is element-wise min and max is element-wise max. 
    """
    print(len(documents))
    matrix = []
    count = 0
    for i in range(len(documents)):
        vector = 0
        if method.lower() == 'avg':
            vector = avg_docvec(documents[i],embeddings)
        elif method.lower() == 'min':
            vector = min_docvec(documents[i],embeddings)
        elif method.lower() == 'max':
            vector = max_docvec(documents[i],embeddings)
        else:
            print("Please enter method argument as min, max or avg")
            return
        if i == 0:
            matrix = vector
        else:
            try:
                matrix = np.column_stack((matrix,vector)) # Concat all vectors into a matrix of order (300,N of docs)
            except:
                count += 1
    matrix = matrix.reshape((len(documents),300)) # For sklearn, reshape the matrix into order (N of docs,300), so rows = docs
    return matrix

articles_matrix = docs_to_matrix(documents,embeddings,'avg')     

1604
COUNT:  0


Here we make the training and test datasets using the article representations and predictions. Then we create a logistic regression object, fit the model on the training data and check it on the test info.

#### Notes on testing: 
* A majority classifier gets outcome Precision = 0.62, Recall = 0.78, F = 0.69. Unregularized tf-idf acts as a majority classifier.
* Using regularized tf-idf logreg (C = 100), Precision/Recall/F =  0.82, 0.80, 0.73. This is no longer a majority classifier. 
* Representing articles as elementwise average of the word embeddings does not change the model from being a majority classifier, even when using extreme regularization (C = .0001). This applies for min and max document vectors as well. 

In [62]:
# # Use the TF-IDF representation of each article as a feature matrix
# sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
# articles_matrix = sklearn_tfidf.fit_transform(documents)
# X_train, X_test, y_train, y_test = train_test_split(articles_matrix, predictions, test_size = .3, random_state=25)

# Use the combination of pre-trained word embeddings in each article as a feature matrix
X_train, X_test, y_train, y_test = train_test_split(articles_matrix, predictions, test_size = .3, random_state=25)

In [63]:
# Logistic Regression
logreg = LogisticRegressionCV(penalty='l2', scoring="f1",Cs=[.00001,.0001,.001,.01,.1,.2,.5,.8,1,2,5,10,100,1000])
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print(y_pred)

/Users/Tim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 1 3 3 1 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 0 3
 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 0 3 3 3 3 3 3
 1 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1
 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1
 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 0 3 3 3 3 3 2 3
 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 0 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3
 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 1 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


#### Model Evaluation: 
Here we evaluate the effectiveness of our model using a confusion matrix, precision, recall and F1-score

In [64]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print(classification_report(y_test, y_pred))

[[  3   0   1  53]
 [  2   4   0  24]
 [  0   2   0  15]
 [ 14   5   4 355]]
             precision    recall  f1-score   support

          0       0.16      0.05      0.08        57
          1       0.36      0.13      0.20        30
          2       0.00      0.00      0.00        17
          3       0.79      0.94      0.86       378

avg / total       0.66      0.75      0.70       482



In [102]:
# CNN Implementation based off Keras models

# Define hyperparameters
MAX_SEQUENCE_LENGTH = 700 # Semi-arbitrary, not sure how to tune this. Articles range from 6 - 6500 words with mean of 659
VALIDATION_SPLIT = 0.2 
EMBEDDING_DIM = 300 # Google News embeddings are 300 dimensional
l2_strength = 2.0
FILTERS = 300 # Bear with me, trying to change batch size from arbitrary 128 to something meaningful
DROPOUT = 0.8 # Dropout strength 

# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(documents)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(documents)

# pad our doc sequences to a max length of MAX_SEQUENCE_LENGTH words
data = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Re-using the list of integer labels generated earlier, make a binary class matrix
labels = to_categorical(np.asarray(predictions))
possibilities = ['mixture of true and false', 'mostly false', 'no factual content', 'mostly true']
# Split into train and test sets - I try 2 versions, one from Keras and one using sklearn functions

## sklearn version of train and test split
# x_train, x_test, y_train, y_test = train_test_split(data, predictions, test_size = 0.3, random_state=25) 

# Keras blog post version of train-test split
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]


# Note I skip a step here - embeddings already loaded into memory as KeyedVectors above

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = None
    try:
        embedding_vector = embeddings[word] # Get the vector for a given word
    except:
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector        

In [103]:
# Create the embedding layer from the embedding matrix
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Create the model
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') 
embedded_sequences = embedding_layer(sequence_input) 

# x = Conv1D(FILTERS, 5, activation='relu',kernel_regularizer=l2(l2_strength))(embedded_sequences) 
# x = MaxPooling1D(5)(x)
# x = Conv1D(FILTERS, 5, activation='relu',kernel_regularizer=l2(l2_strength))(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(FILTERS, 5, activation='relu',kernel_regularizer=l2(l2_strength))(x)
# x = MaxPooling1D(23)(x)  # Best size - 23
# x = Flatten()(x)
# x = Dense(FILTERS, activation='relu')(x)
# preds = Dense(len(possibilities), activation='softmax')(x)

x = Conv1D(FILTERS, 5, activation='relu')(embedded_sequences) 
x = MaxPooling1D(5)(x)
x = Dropout(DROPOUT)(x)
x = Conv1D(FILTERS, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Dropout(DROPOUT)(x)
x = Conv1D(FILTERS, 5, activation='relu')(x)
x = MaxPooling1D(23)(x)  # Best size - 23
x = Dropout(DROPOUT)(x)
x = Flatten()(x)
x = Dense(FILTERS, activation='relu')(x)
preds = Dense(len(possibilities), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# for layer in model.layers: # This is debugging code - do not include in the final model.
#     print(layer)
#     print("INPUT: ",layer.input_shape," OUTPUT: ",layer.output_shape)

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=3, batch_size=FILTERS)

# Validation best w/ L2 = 79.69%, epochs = 3, max_seq = 700, l2 = 2.0, validation = 0.2, filters = 128
# Validation best w/ dropout = 81.25%, epochs = 3, max_seq = 700, Dropout = 0.5, validation = 0.2, filters = 300


Train on 1284 samples, validate on 320 samples
Epoch 1/3
1284/1284 [==============================] - 91s 71ms/step - loss: 3.8144 - acc: 0.6246 - val_loss: 3.8445 - val_acc: 0.7563
Epoch 2/3
1284/1284 [==============================] - 65s 51ms/step - loss: 3.4170 - acc: 0.7843 - val_loss: 3.8427 - val_acc: 0.7563
Epoch 3/3
1284/1284 [==============================] - 63s 49ms/step - loss: 3.4156 - acc: 0.7843 - val_loss: 3.8378 - val_acc: 0.7563
